# Data Preparation

Bringing the data into the desired format, where each match is represented by two rows with home and away team switched.

**1 - Imports and reading csv files**

In [44]:
#imports
import pandas as pd
import numpy as np

In [60]:
#Read the train.csv and test.csv data ()
train = pd.read_csv("train.csv", encoding = "ISO-8859-1")

#test.csv has rows wrapped in "", which makes parsing a bit difficult. 
#quoting = 3 ignores all quotes and we can get rid of them manually
test = pd.read_csv("test.csv", encoding="ISO-8859-1", quoting=3)

In [61]:
#Check out the data frames
train.head()

,Unnamed: 0,eventBegin,paarung,score,RefOdd1,RefOddX,RefOdd2,RefOddOver
0,1,2015-01-30 20:35:32,VfL Wolfsburg - Bayern München,4:1 (2:0),7.49272,4.85413,1.51180,1.69718
1,2,2015-01-31 15:30:19,SC Freiburg - Eintracht Frankfurt,4:1 (0:1),2.97516,3.94175,2.42760,1.70712
2,3,2015-01-31 15:30:56,Hamburger SV - 1.FC Köln,0:2 (0:0),2.54718,3.33901,3.26786,2.59439
3,4,2015-01-31 15:31:11,Schalke 04 - Hannover 96,1:0 (1:0),2.07487,3.81712,3.88638,1.82383
4,5,2015-01-31 15:31:21,FSV Mainz 05 - SC Paderborn,5:0 (1:0),2.00676,3.76686,4.25880,1.87892


In [62]:
#manually get rid of quotes
test.columns = train.columns
for col in test.select_dtypes(include="object"):
    test[col] = test[col].str.strip("\"")

In [63]:
test.head()

,Unnamed: 0,eventBegin,paarung,score,RefOdd1,RefOddX,RefOdd2,RefOddOver
0,1467,2019-10-25 20:30:06,FSV Mainz 05 - 1.FC Köln,NaN,NaN,NaN,NaN,NA
1,1468,2019-10-26 15:30:00,SC Freiburg - RB Leipzig,NaN,NaN,NaN,NaN,NA
2,1469,2019-10-26 15:30:35,SC Paderborn - Fortuna Düsseldorf,NaN,NaN,NaN,NaN,NA
3,1470,2019-10-26 15:30:42,Bayern München - FC Union Berlin,NaN,NaN,NaN,NaN,NA
4,1471,2019-10-26 15:31:02,Hertha BSC - 1899 Hoffenheim,NaN,NaN,NaN,NaN,NA


**2 - Parsing both teams from "paarung" variable**

In [64]:
#defining a couple of functions
def home_team(row):
    team = row["paarung"].split("-")[0]
    team = team.strip()
    return team

def away_team(row):
    team = row["paarung"].split("-")[1]
    team = team.strip()
    return team

In [65]:
#Apply both functions to the train and test df
train["team1"] = train.apply(lambda row: home_team(row), axis = 1)
train["team2"] = train.apply(lambda row: away_team(row), axis = 1)

In [66]:
#test df
test["team1"] = test.apply(lambda row: home_team(row), axis = 1)
test["team2"] = test.apply(lambda row: away_team(row), axis = 1)

In [67]:
#check unique values of team1 and team2
train["team1"].unique()

array(['VfL Wolfsburg', 'SC Freiburg', 'Hamburger SV', 'Schalke 04',
       'FSV Mainz 05', 'VfB Stuttgart', 'Bayer Leverkusen',
       'Werder Bremen', 'FC Augsburg', 'Eintracht Frankfurt',
       'Hannover 96', "Bor. M'gladbach", 'Bayern München',
       '1899 Hoffenheim', 'Paderborn', 'Hertha BSC', '1.FC Köln',
       'Bor. Dortmund', 'Wolfsburg', 'FSV Mainz', "B. M'gladbach",
       'Borussia Dortmund', 'SC Paderborn', "Borussia M'gladbach",
       'Mainz 05', 'Eint. Frankfurt', 'Hoffenheim', 'Bay. München',
       "M'gladbach", 'B. Dortmund', 'Karlsruher SC', 'SV Darmstadt 98',
       'FC Ingolstadt', 'Ingolstadt', 'Ein. Frankfurt', 'Augsburg',
       'E. Frankfurt', '1. FC Nürnberg', 'RB Leipzig', 'Freiburg',
       'E. Braunschweig', 'Holstein Kiel', 'Fortuna Düsseldorf',
       'B. Leverkusen', 'Fort. Düsseldorf', 'Eintr. Frankfurt',
       'F. Düsseldorf', 'FC Union Berlin', 'Union Berlin'], dtype=object)

In [68]:
train["team2"].unique()

array(['Bayern München', 'Eintracht Frankfurt', '1.FC Köln',
       'Hannover 96', 'SC Paderborn', "Borussia M'gladbach",
       'Borussia Dortmund', 'Hertha BSC', '1899 Hoffenheim',
       'VfL Wolfsburg', 'FSV Mainz 05', 'SC Freiburg', 'Schalke 04',
       'Werder Bremen', 'Hamburger SV', 'Bayer Leverkusen',
       'VfB Stuttgart', 'FC Augsburg', 'Hoffenheim', 'Bor. Dortmund',
       'FSV Mainz', 'Wolfsburg', "Bor. M'gladbach", 'Bay. München',
       "B. M'gladbach", 'Eintr. Frankfurt', 'Augsburg', 'FC Köln',
       'Tabellendritter 2.BL', 'Karlsruher SC', 'FC Ingolstadt',
       'E. Frankfurt', 'Darmstadt 98', 'TSG 1899 Hoffenheim',
       'SV Darmstadt 98', 'Bayern Munich', 'Ein. Frankfurt',
       '1 FSV Mainz 05', 'Eint. Frankfurt', 'Ingolstadt',
       '1. FC Nürnberg', 'RB Leipzig', 'E. Braunschweig', 'Holstein Kiel',
       'Fortuna Düsseldorf', 'B. Dortmund', 'B. Leverkusen',
       'Union Berlin', 'FC Union Berlin', 'F. Düsseldorf'], dtype=object)

In [69]:
#team2 contains the unspecified "Tabellendritter 2 BL" - drop those rows
train = train.loc[train.team2 != "Tabellendritter 2.BL", ]

In [70]:
#a lot of variability in naming of same teams - clean that up with a couple of steps

#function that returns the longer element of the team name (usually the city name, but for a few exceptions..)
def clean_name(row, team):
    
    name = max(row[team].split(), key=len)
    name = name.strip()
    #handle exceptions "Eintracht Frankfurt" "1.FC Köln" and "Borussia Dortmund" and "Bayern Munich"
    if name == "Eintracht":
        name = "Frankfurt"
    elif name == "1.FC":
        name = "Köln"
    elif name ==  "Borussia":
        name = "Dortmund"
    elif name == "Bayern":
        name = "München"
        
    
    return name

In [71]:
#Apply both functions to the train and test df
train["team1"] = train.apply(lambda row: clean_name(row, "team1"), axis = 1)
train["team2"] = train.apply(lambda row: clean_name(row, "team2"), axis = 1)

In [72]:
#test df
test["team1"] = test.apply(lambda row: clean_name(row, "team1"), axis = 1)
test["team2"] = test.apply(lambda row: clean_name(row, "team2"), axis = 1)

**3 - Parsing scores from "score" - only for train (obviously)**

In [74]:
train.loc[:, "score1"] = train.loc[:, "score"].str[0]
train.loc[:, "score2"] = train.loc[:, "score"].str[2]

In [76]:
#add empty score1 and score2 columns to test df - to keep the same shape and format
test.loc[:, "score1"] = None
test.loc[:, "score2"] = None

**4 - bring dataframe in required format**

In [79]:
#for now still with one row per match
train = train.loc[:, ["eventBegin", "team1", "team2", "score1", "score2", "RefOdd1", "RefOddX", "RefOdd2", "RefOddOver"]]
test = test.loc[:, ["eventBegin", "team1", "team2", "score1", "score2", "RefOdd1", "RefOddX", "RefOdd2", "RefOddOver"]]

In [80]:
#add a "home" variable and set it to 1
train["home"] = 1
test["home"] = 1

In [85]:
#create alternative dataframes that swap team1 and 2, score1 and 2 and RefOdd1 and 1 and add "home" = 0
train_alt = pd.concat([train.eventBegin, train.team2, train.team1, 
                          train.score2, train.score1, train.RefOdd2, train.RefOddX, train.RefOdd1, 
                          train.RefOddOver], axis=1)
train_alt["home"] = 0

test_alt = pd.concat([test.eventBegin, test.team2, test.team1, 
                          test.score2, test.score1, test.RefOdd2, test.RefOddX, test.RefOdd1, 
                          test.RefOddOver], axis=1)
test_alt["home"] = 0

In [88]:
train_alt.head()

,eventBegin,team1,team2,score1,score2,RefOdd1,RefOddX,RefOdd2,RefOddOver,home
0,2015-01-30 20:35:32,München,Wolfsburg,1,4,1.51180,4.85413,7.49272,1.69718,0
1,2015-01-31 15:30:19,Frankfurt,Freiburg,1,4,2.42760,3.94175,2.97516,1.70712,0
2,2015-01-31 15:30:56,Köln,Hamburger,2,0,3.26786,3.33901,2.54718,2.59439,0
3,2015-01-31 15:31:11,Hannover,Schalke,0,1,3.88638,3.81712,2.07487,1.82383,0
4,2015-01-31 15:31:21,Paderborn,Mainz,0,5,4.25880,3.76686,2.00676,1.87892,0


In [89]:
#rename columns
train_alt.columns = train.columns
test_alt.columns = test.columns

In [109]:
#combine the two dfs each with concat and then sort by index and home (make index explicit)
train_compl = pd.concat([train, train_alt], axis = 0)
test_compl = pd.concat([test, test_alt], axis = 0)

In [110]:
#reset index
train_compl = train_compl.reset_index()

In [111]:
#sort
train_compl = train_compl.sort_values(by=["index", "home"], ascending=[True, False])
#drop old index column
train_compl = train_compl.drop(columns= "index")
#reset index again (no new column)
train_compl = train_compl.reset_index(drop=True)

In [113]:
#repeat for test df
test_compl = test_compl.reset_index()
test_compl = test_compl.sort_values(by=["index", "home"], ascending=[True, False])
test_compl = test_compl.drop(columns="index")
test_compl = test_compl.reset_index(drop=True)

**One last glance and overview over dfs**

In [127]:
#the team names are only the city names and not the "proper" club names - let's replace them

#make a dictionary from the raw_names and the proper names

#keys
raw_names = train_compl.team1.unique()
#values
proper_names = ["VfL Wolfsburg", "Bayern München", "SC Freiburg", "Eintracht Frankfurt", "Hamburger SV", "1.FC Köln",
             "Schalke 04", "Hannover 96", "Mainz 05", "Paderborn SC", "VfB Stuttgart", "Borussia M'gladbach",
             "Bayer Leverkusen", "Borussia Dortmund", "Werder Bremen", "Hertha BSC", "FC Augsburg", "TSG Hoffenheim", 
             "Karlsruher SC", "FC Ingolstadt", "SV Darmstadt", "FC Nürnberg", "RB Leipzig", "Eintracht Braunschweig",
             "Holstein Kiel", "Fortuna Düsseldorf", "Union Berlin"]

In [128]:
#set up dict
team_dict = {}
for i in range(len(raw_names)):
    team_dict[raw_names[i]] = proper_names[i]

In [130]:
#replace
train_compl = train_compl.replace({"team1": team_dict, "team2": team_dict})
test_compl = test_compl.replace({"team1": team_dict, "team2": team_dict})

In [120]:
train_compl.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2928 entries, 0 to 2929
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   eventBegin  2928 non-null   object 
 1   team1       2928 non-null   object 
 2   team2       2928 non-null   object 
 3   score1      2928 non-null   object 
 4   score2      2928 non-null   object 
 5   RefOdd1     2902 non-null   float64
 6   RefOddX     2902 non-null   float64
 7   RefOdd2     2902 non-null   float64
 8   RefOddOver  2902 non-null   float64
 9   home        2928 non-null   int64  
dtypes: float64(4), int64(1), object(5)
memory usage: 251.6+ KB


In [119]:
#two missing rows for score - drop these
train_compl = train_compl.loc[~train_compl["score1"].isna(), ]

In [124]:
#test df
test_compl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   eventBegin  18 non-null     object 
 1   team1       18 non-null     object 
 2   team2       18 non-null     object 
 3   score1      0 non-null      object 
 4   score2      0 non-null      object 
 5   RefOdd1     0 non-null      float64
 6   RefOddX     0 non-null      float64
 7   RefOdd2     0 non-null      float64
 8   RefOddOver  0 non-null      object 
 9   home        18 non-null     int64  
dtypes: float64(3), int64(1), object(6)
memory usage: 1.5+ KB


In [123]:
#test dfs RefOddOver is actually treated as a character column
test_compl["RefOddOver"] = None

**Final cleaned up and reformatted dataframe of the train data**

In [132]:
train_compl.head()

,eventBegin,team1,team2,score1,score2,RefOdd1,RefOddX,RefOdd2,RefOddOver,home
0,2015-01-30 20:35:32,VfL Wolfsburg,Bayern München,4,1,7.49272,4.85413,1.51180,1.69718,1
1,2015-01-30 20:35:32,Bayern München,VfL Wolfsburg,1,4,1.51180,4.85413,7.49272,1.69718,0
2,2015-01-31 15:30:19,SC Freiburg,Eintracht Frankfurt,4,1,2.97516,3.94175,2.42760,1.70712,1
3,2015-01-31 15:30:19,Eintracht Frankfurt,SC Freiburg,1,4,2.42760,3.94175,2.97516,1.70712,0
4,2015-01-31 15:30:56,Hamburger SV,1.FC Köln,0,2,2.54718,3.33901,3.26786,2.59439,1


# Model selection

We are dealing with a multi-class classification problem, as there are three possible outcomes: home win, draw, away win.
Since the only data we have available in the test data (the upcoming match day) are the teams involved in the matches we will need to rely on the past outcomes of those teams to predict the outcomes/the outcome probabilities.

I would suggest using Gradient Boosting as a modelling algorithm - an ensemble method combining multiple decision trees. It handles multi class classification well and typically outperforms for example random forests.

# Fitting a model

**1 - prepping the data for fitting**

In [133]:
# first step: create an outcome variable by comparing the scores 
def match_result(row):
    if row["score1"] == row["score2"]:
            return "draw"
    
    elif row["home"] == 1:
    
        if row["score1"] > row["score2"]:
            return "home"
    
        elif row["score1"] < row["score2"]:
            return "away"
    
    elif row["home"] == 0:
        
        if row["score1"] < row["score2"]:
            return "home"
        
        elif row["score1"] > row["score2"]:
            return "away"

In [134]:
train_compl["result"] = train_compl.apply(lambda row: match_result(row), axis = 1)

In [135]:
train_compl.head()

,eventBegin,team1,team2,score1,score2,RefOdd1,RefOddX,RefOdd2,RefOddOver,home,result
0,2015-01-30 20:35:32,VfL Wolfsburg,Bayern München,4,1,7.49272,4.85413,1.51180,1.69718,1,home
1,2015-01-30 20:35:32,Bayern München,VfL Wolfsburg,1,4,1.51180,4.85413,7.49272,1.69718,0,home
2,2015-01-31 15:30:19,SC Freiburg,Eintracht Frankfurt,4,1,2.97516,3.94175,2.42760,1.70712,1,home
3,2015-01-31 15:30:19,Eintracht Frankfurt,SC Freiburg,1,4,2.42760,3.94175,2.97516,1.70712,0,home
4,2015-01-31 15:30:56,Hamburger SV,1.FC Köln,0,2,2.54718,3.33901,3.26786,2.59439,1,away


**For fitting I will only use the home == 1 rows but then use both team1 and team2 as features for the model**

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

In [143]:
#prepare the columns for encoding
X_train = np.array(train_compl.loc[train_compl.home == 1, ["team1", "team2"]])

In [144]:
#set up an encoder instance
ohe = OneHotEncoder(sparse_output=False)

In [145]:
#fit and transform the feature variables
X_train = ohe.fit_transform(X_train)

In [172]:
#now apply same encoding to X_test
X_test = np.array(test_compl.loc[test_compl.home == 1, ["team1", "team2"]])

In [173]:
X_test = ohe.transform(X_test)

In [ ]:
# Label Encode the outcome variable

In [149]:
#encoder instance
lab_enc = LabelEncoder()

In [169]:
y_train = np.array(train_compl.loc[train_compl.home == 1, "result"])

In [170]:
y_train = lab_enc.fit_transform(y_train)

**2 - Fitting the model**

In [155]:
from sklearn.ensemble import GradientBoostingClassifier

In [156]:
#set up model instance with default hyperparameters
mod = GradientBoostingClassifier()

In [171]:
#and fit the model
mod.fit(X_train, y_train)

GradientBoostingClassifier()

**3 - Predict probabilities for X_test**

In [176]:
#predict 
probs_predicted = mod.predict_proba(X_test)

In [188]:
probs_predicted

array([[0.46877033, 0.29141752, 0.23981215],
       [0.41082921, 0.22077229, 0.3683985 ],
       [0.11879092, 0.16074306, 0.72046603],
       [0.54808491, 0.40684763, 0.04506746],
       [0.41058378, 0.35616195, 0.23325427],
       [0.26096237, 0.33457843, 0.4044592 ],
       [0.46031006, 0.25442209, 0.28526786],
       [0.43622631, 0.31595179, 0.2478219 ],
       [0.64988602, 0.12259908, 0.2275149 ]])

In [178]:
#rearrange columns in array from [away, draw, home] to [home, draw, away] like in the dataframes
probs_predicted = probs_predicted[:, [2,1,0]]

In [186]:
#fill in the missing odds in test df
test_compl.loc[test_compl.home==1, ["RefOdd1", "RefOddX", "RefOdd2"]] = 1/probs_predicted
#switched order for home == 0
test_compl.loc[test_compl.home==0, ["RefOdd1", "RefOddX", "RefOdd2"]] = 1/probs_predicted[:, [2,1,0]]

**Predicted (inverted) probabilities in the test dataframe**

In [189]:
test_compl

,eventBegin,team1,team2,score1,score2,RefOdd1,RefOddX,RefOdd2,RefOddOver,home
0,2019-10-25 20:30:06,Mainz 05,1.FC Köln,None,None,2.133241,3.431503,4.169931,None,1
1,2019-10-25 20:30:06,1.FC Köln,Mainz 05,None,None,4.169931,3.431503,2.133241,None,0
2,2019-10-26 15:30:00,SC Freiburg,RB Leipzig,None,None,2.434101,4.529554,2.714452,None,1
3,2019-10-26 15:30:00,RB Leipzig,SC Freiburg,None,None,2.714452,4.529554,2.434101,None,0
4,2019-10-26 15:30:35,Paderborn SC,Fortuna Düsseldorf,None,None,8.418152,6.221109,1.387990,None,1
5,2019-10-26 15:30:35,Fortuna Düsseldorf,Paderborn SC,None,None,1.387990,6.221109,8.418152,None,0
6,2019-10-26 15:30:42,Bayern München,Union Berlin,None,None,1.824535,2.457923,22.188957,None,1
7,2019-10-26 15:30:42,Union Berlin,Bayern München,None,None,22.188957,2.457923,1.824535,None,0
8,2019-10-26 15:31:02,Hertha BSC,TSG Hoffenheim,None,None,2.435557,2.807712,4.287167,None,1
9,2019-10-26 15:31:02,TSG Hoffenheim,Hertha BSC,None,None,4.287167,2.807712,2.435557,None,0


# Possible Performance Metrics

As we are interested in predicting probabilities (to infer betting odds) of all outcomes rather than predicting the exact outcome, measures such as accuracy, precision and recall (or F1 score, combining the latter two) are not entirely suited for out case.

Log-loss and the Brier Score are more informative in our use case as they give a score based on the predicted probabilities vs. the true outcomes.

# Model improvements and suggestions

The current model only considers the overall performance of a team and the specific match up performance of a specific team vs another team, as these are the data that are available. This is obviously a somewhat crude approach. First off, a team's most recent performance is more predictive of current or future outcomes than performance further back in time. Giving more recent matches higher predictive weight would therefore probably lead to better predictions.

There are further details that could be included if the data is available such as the specific players that are available in any given match or if key contributors to the team are injured, or suspended for example. In combination with individual player performance data this can enable more precise 

A lot of these details are presumably included implicitly in the Reference Odds of the past games. While these odds are not available for future games, as they are more or less the goal of the prediction, we cannot directly take them into account. But it might be possible to use some of the information contained in the odds to improve the predictions based on the specific match ups of the current match day.

On a more technical note: the match results have an imbalanced nature, due to an apparent homefield advantage. This can cause problems for classification models, as the model can be biased towards predicting the majority class. Given enough data one might be able to afford to downsample towards a more balanced dataset. Another approach could be to use a model that allows for differential weighting in the fitting process. As I understand it, the XGboost classifier provides this possibility. 